### 1. Read the images and generate the train and test dataset (5 points)

In [0]:
from google.colab import drive

In [32]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [0]:
from zipfile import ZipFile
with ZipFile('plant-seedlings-classification.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile('test.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile('train.zip', 'r') as z:
  z.extractall()

In [0]:
# extract the test data
x_test=[]
os.chdir('/content/drive/My Drive/Colab Notebooks/test')
import cv2
for i in os.listdir():
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128)) #resize to have all the images of same size
    x_test.append(dummy)
    x_test.append(i)


In [0]:
# split the data
x_train = []
y_train = []

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [41]:
ls

plant-seedlings-classification.zip  test/                             train.zip
R7_Project1_Computer_Vision.ipynb   test.zip                          yelp.csv
sample_submission.csv               textClassificationExercise.ipynb
SVHN_single_grey1.h5                train/


In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks/train')

In [43]:
os.listdir()

['Fat Hen',
 'Small-flowered Cranesbill',
 'Cleavers',
 'Black-grass',
 'Sugar beet',
 'Shepherds Purse',
 'Charlock',
 'Loose Silky-bent',
 'Scentless Mayweed',
 'Maize',
 'Common Chickweed',
 'Common wheat']

In [44]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/Colab Notebooks/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)


Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock
Loose Silky-bent
Scentless Mayweed
Maize
Common Chickweed
Common wheat


In [45]:
print("Shape of X_train:",x_train[0].shape)
print("No of images in X_train",len(x_train))

Shape of X_train: (128, 128, 3)
No of images in X_train 4750


In [0]:
import pandas as pd
dum = pd.get_dummies(y_train)

In [0]:
encoded_labels = dum
y_train = dum

In [0]:
import numpy as np
y_train = np.array(y_train)

In [0]:
x_train = np.array(x_train)

In [50]:
x_train[0].shape

(128, 128, 3)

### 3. Initialize & build the model (10 points)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print ("x_train : ", len(X_train))
print ("x_val : ",len(X_val))

x_train :  3800
x_val :  950


In [55]:
X_train[0].shape

(128, 128, 3)

In [0]:
X_train = x_train.reshape(x_train.shape[0],128,128,3)

In [0]:
X_val = X_val.reshape(X_val.shape[0],128,128,3)

In [58]:
print (X_train.shape)
print (X_val.shape)

(4750, 128, 128, 3)
(950, 128, 128, 3)


In [59]:
print(Y_train.shape)
print(Y_val.shape)

(3800, 12)
(950, 12)


In [0]:
Y_train = np.array(y_train)
X_train = np.array(X_train)

In [0]:
# build model

from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense

In [0]:
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

In [63]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax')) 


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 128)       0         
__________

In [98]:
batch_size = 64
nb_epochs = 10
history = model.fit(X_train,Y_train,
                    epochs=20, 
                    validation_data=(X_val,Y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 4750 samples, validate on 950 samples
Epoch 1/20
4750/4750 [==============================] - 12s 3ms/step - loss: 1.3968 - acc: 0.5116 - val_loss: 1.0422 - val_acc: 0.6505
Epoch 2/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.9753 - acc: 0.6646 - val_loss: 0.7877 - val_acc: 0.7526
Epoch 3/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.7751 - acc: 0.7404 - val_loss: 0.6587 - val_acc: 0.7989
Epoch 4/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.6246 - acc: 0.7918 - val_loss: 0.5779 - val_acc: 0.8105
Epoch 5/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.5054 - acc: 0.8213 - val_loss: 0.4582 - val_acc: 0.8421
Epoch 6/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.4142 - acc: 0.8568 - val_loss: 0.2918 - val_acc: 0.9032
Epoch 7/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.3297 - acc: 0.8838 - val_loss: 0.2337 - val_acc: 0.9189
Epoch 8/

### 4. Optimize the model (5 points)

### Observation: The above model has already been implemented with the best optimizer and activations. Anyhow we will try changing epoch , optimiser ,dropouts and batch size.

In [0]:
from keras.layers import LeakyReLU

In [0]:
model1 = Sequential()
model1.add(BatchNormalization(input_shape = (128,128,3)))
model1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D(pool_size=2))
model1.add(Dropout(0.25))

model1.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D(pool_size=2))
model1.add(Dropout(0.2))

model1.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model1.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model1.add(Flatten()) 

# fully connected layer
model1.add(Dense(units=128,activation = 'relu'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(Dense(units = 64, activation = 'relu'))
model1.add(LeakyReLU(alpha=0.1))
#model.add(Dropout(0.3))
model1.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.3))
model1.add(Dense(units = 12, activation = 'softmax')) 

In [0]:
#optimizer = adam(lr=0.001)
model1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [78]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 63, 63, 64)        32832     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 63, 63, 64)        0         
__________

In [81]:
batch_size = 78
nb_epochs = 10
history = model1.fit(x_train,y_train,
                    epochs=20, 
                    validation_data=(X_val,Y_val),
                    verbose = 1,
                    initial_epoch=0)

Train on 4750 samples, validate on 950 samples
Epoch 1/20
4750/4750 [==============================] - 13s 3ms/step - loss: 1.8722 - acc: 0.3766 - val_loss: 1.3877 - val_acc: 0.5326
Epoch 2/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.9958 - acc: 0.6634 - val_loss: 0.9409 - val_acc: 0.6758
Epoch 3/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.7076 - acc: 0.7552 - val_loss: 0.6402 - val_acc: 0.7895
Epoch 4/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.5574 - acc: 0.8080 - val_loss: 0.4517 - val_acc: 0.8442
Epoch 5/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.4304 - acc: 0.8520 - val_loss: 0.3287 - val_acc: 0.8853
Epoch 6/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.3418 - acc: 0.8787 - val_loss: 0.2507 - val_acc: 0.9137
Epoch 7/20
4750/4750 [==============================] - 12s 3ms/step - loss: 0.3131 - acc: 0.8863 - val_loss: 0.2203 - val_acc: 0.9305
Epoch 8/

### 5. Predict the accuracy for both train and validation data (5 points)

In [0]:
predict_train = model.predict(X_train)
predict_val = model.predict(X_val)

In [100]:
from sklearn import metrics
print ("Train_accuracy: ", metrics.accuracy_score(np.argmax(y_train, axis=1), np.argmax(predict_train, axis=1)))

Train_accuracy:  0.9922105263157894


In [101]:
from sklearn.metrics import classification_report

print ("CR for train data")
print(classification_report(np.argmax(y_train, axis=1), np.argmax(predict_train, axis=1)))

CR for train data
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       263
           1       0.99      0.99      0.99       390
           2       1.00      0.99      0.99       287
           3       1.00      1.00      1.00       611
           4       1.00      1.00      1.00       221
           5       1.00      1.00      1.00       475
           6       0.97      0.99      0.98       654
           7       1.00      1.00      1.00       221
           8       1.00      1.00      1.00       516
           9       1.00      0.99      1.00       231
          10       1.00      1.00      1.00       496
          11       0.98      1.00      0.99       385

   micro avg       0.99      0.99      0.99      4750
   macro avg       0.99      0.99      0.99      4750
weighted avg       0.99      0.99      0.99      4750



In [104]:
from sklearn import metrics
print ("val_accuracy: ", metrics.accuracy_score(np.argmax(Y_val, axis=1), np.argmax(predict_val, axis=1)))

val_accuracy:  0.9894736842105263


In [107]:

print ("CR for val data")
print(classification_report(np.argmax(Y_val, axis=1), np.argmax(predict_val, axis=1)))

CR for val data
              precision    recall  f1-score   support

           0       0.96      0.85      0.90        52
           1       1.00      1.00      1.00        70
           2       1.00      1.00      1.00        50
           3       1.00      1.00      1.00       118
           4       1.00      1.00      1.00        47
           5       1.00      1.00      1.00       102
           6       0.94      0.99      0.96       134
           7       1.00      1.00      1.00        40
           8       1.00      1.00      1.00        90
           9       1.00      1.00      1.00        49
          10       1.00      1.00      1.00       112
          11       1.00      1.00      1.00        86

   micro avg       0.99      0.99      0.99       950
   macro avg       0.99      0.99      0.99       950
weighted avg       0.99      0.99      0.99       950



### Observation: F1 score of class 1 and 6 is less in val accuracy may be due to inconsistency in the class distribution.